In [6]:
import pandas as pd
import numpy as np

dft = pd.read_csv("complete_times.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("complete_distances.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [7]:
def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)
def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

In [8]:
table=pd.read_csv("route.csv", delimiter=";")
time_table=pd.read_csv("route_time.csv", delimiter=";")

In [9]:
import datetime
def difference(day, index):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index+1], '%H:%M')
    return ((time2-time).seconds)/60

In [41]:
gained_time1=0
gained_time2=0 
days = ['martedi', 'mercoledi','giovedì']
#for day in days:
day='martedi'
for i in range(len(time_table[day].dropna())-2):
    try:
        diff=difference(day, i)
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][i+1]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time1 :
            gained_time2+=diff
            gained_time1-=ctime
            print(f"{day}: primo : {paese} : {gained_time1} : {gained_time2}")
        elif (diff+(diff/20))>ctime-gained_time2:
            gained_time1+=diff
            gained_time2-=ctime
            print(f"{day}: secondo : {paese} : {gained_time1} : {gained_time2}")
        else:
            print(f"{day}: impossible : {paese} : {gained_time1} : {gained_time2} : {diff}")
            gained_time1=(gained_time1-(diff-float(value.values[0][0])))+diff
            gained_time2=(gained_time2-(diff-float(value.values[0][0])))+diff        
    except:
        print(value)
    #if diff<dft[row[paese].values]:
    #    print("Not acceptable!")

martedi: primo : Cremona, Felice Cavallotti, 2 : -33.4 : 90.0
martedi: secondo : Casalpusterlengo : -18.4 : 55.9
martedi: secondo : Cremona, Ugolani Dati, 4 : -3.3999999999999986 : 21.35
martedi: impossible : Crema, Civerchi : -3.3999999999999986 : 21.35 : 15.0
martedi: secondo : Bonemerse : 42.083333333333336 : 16.450000000000003
martedi: impossible : Sospiro : 42.083333333333336 : 16.450000000000003 : 10.0
martedi: primo : Vailate : 40.78333333333332 : 139.53333333333333
martedi: impossible : Pianengo : 40.78333333333332 : 139.53333333333333 : 20.0
martedi: primo : Vescovato : 80.41666666666663 : 464.83333333333337
martedi: primo : Offanengo : 30.566666666666627 : 474.83333333333337
martedi: secondo : Piadena : 35.56666666666663 : 409.58333333333337
martedi: primo : Gussola : 17.383333333333294 : 429.58333333333337
martedi: primo : Solarolo : 8.29999999999996 : 594.5833333333334
martedi: secondo : Ostiano : 8.29999999999996 : 570.95
martedi: primo : Scandolara Ravara, Italia : -20.01

In [107]:
diff=difference("martedi", 15)
row=dft[get_row(dft, table["martedi"][0]).values]
paese=table["martedi"][15]
value=row.loc[:,get_column(dft, paese).values]
#gained_time=gained_time+(diff-float(value.values[0][0]))

In [40]:
diff=difference("lunedi", 0)

row=dft[get_row(dft, table["lunedi"][0]).values]
value=row.loc[:,get_column(dft, "bagnolo").values]
float(value.values[0][0])

dft[get_row(dft, "crema").values]

,"Agnadello, Marconi, 6","Annicco, Giuseppe Garibaldi, 6","Bagnolo Cremasco, Roma, 3","Bonemerse, Roma, 25","Camisano, Trieste, 57","Capergnanica, XI febbraio, 7","Capralba, Piave, 2","Casalbuttano ed Uniti, Jacini, 23","Casale Cremasco-Vidolasco, Roma, 56","Casaletto Vaprio, Maggiore, 7",...,"Spino d'Adda, Circonvallazione, 13","Trescore Cremasco, Vittorio Veneto, 1","Trigolo, Roma, 26","Vaiano Cremasco, Gloriosi Caduti, 3","Vailate, Dante Alighieri, 7","Vescovato, Roma, 44","Cremona, Felice Cavallotti, 2","Cremona, Palosca n. 2",Ostiano,San Giovanni
"Bagnolo Cremasco, Roma, 3",15.150000,"3,22666666666666E+016",0.0,"4,94833333333333E+016",22.9,7.550000,"1,95833333333333E+016",40.350000,"2,18666666666666E+016",14.583333,...,"1,32666666666666E+016","1,21166666666666E+016",24.75,5.983333e+15,"2,15833333333333E+016",52.800000,"4,61666666666666E+016","4,70833333333333E+016",61.85,61.85
"Casale Cremasco-Vidolasco, Roma, 56",21.616667,32.3,22.65,51.016666666666666,6.283333333333333,18.716667,12.933333333333334,32.750000,0.0,12.150000,...,23.866666666666667,12.233333333333333,21.583333333333332,1.988333e+01,17.583333333333332,54.500000,47.81666666666667,48.68333333333333,59.983333333333334,69.41666666666667
"Crema, Civerchi, 9",20.816667,28.333333333333332,13.35,45.38333333333333,16.766666666666666,10.233333,14.266666666666667,35.550000,15.733333333333333,12.733333,...,18.883333333333333,12.583333333333334,18.65,1.350000e+01,19.1,48.816667,42.25,43.03333333333333,57.88333333333333,63.85
"Crema, Dante Alighieri, 49",20.200000,27.466666666666665,12.75,44.583333333333336,17.116666666666667,8.066667,15.733333333333333,35.150000,16.15,14.000000,...,18.216666666666665,13.433333333333334,19.05,1.290000e+01,20.516666666666666,48.050000,41.46666666666667,42.25,57.1,63.06666666666667
"Monte Cremasco, Roma, 12",12.016667,34.416666666666664,8.3,51.86666666666667,23.233333333333334,9.833333,17.45,41.233333,20.366666666666667,12.466667,...,10.866666666666667,9.95,25.966666666666665,3.350000e+00,19.533333333333335,55.200000,48.55,49.35,64.21666666666667,70.03333333333333
"Ripalta Cremasca, Roma, 5",19.683333,23.833333333333332,12.55,41.016666666666666,16.833333333333332,5.600000,18.433333333333334,31.716667,15.85,15.800000,...,17.733333333333334,15.533333333333333,18.583333333333332,1.248333e+01,23.033333333333335,44.483333,37.983333333333334,38.55,53.36666666666667,59.35
"Trescore Cremasco, Vittorio Veneto, 1",12.466667,36.88333333333333,13.983333333333333,54.28333333333333,15.35,14.433333,9.55,36.950000,12.35,4.516667,...,13.566666666666666,0.0,26.283333333333335,9.750000e+00,10.766666666666667,57.650000,51.38333333333333,51.81666666666667,64.26666666666667,72.7
"Vaiano Cremasco, Gloriosi Caduti, 3",13.266667,34.53333333333333,4.9,51.95,23.566666666666666,10.050000,17.766666666666666,41.816667,20.566666666666666,12.700000,...,11.483333333333333,10.2,25.933333333333334,0.000000e+00,19.683333333333334,55.350000,48.833333333333336,49.53333333333333,64.18333333333334,70.33333333333333
